In [1]:
import csv
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from datetime import datetime
from sklearn.model_selection import train_test_split

In [2]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

2022-12-04 22:38:27.116252: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-04 22:38:27.324284: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-12-04 22:38:27.959096: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-12-04 22:38:27.959151: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not l

Num GPUs Available:  0


2022-12-04 22:38:28.644440: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_SYSTEM_DRIVER_MISMATCH: system has unsupported display driver / cuda driver combination
2022-12-04 22:38:28.644492: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: danilo-Nitro-AN515-58
2022-12-04 22:38:28.644530: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: danilo-Nitro-AN515-58
2022-12-04 22:38:28.644725: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 510.108.3
2022-12-04 22:38:28.644759: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 525.60.11
2022-12-04 22:38:28.644766: E tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:313] kernel version 525.60.11 does not match DSO version 510.108.3 -- cannot find working devices in this co

In [3]:
train_data = pd.read_csv("data/train.csv")

In [4]:
# new features
train_data["url_count"] = train_data["urls"].apply(lambda s: s[1:-1].count("\'")/2)
train_data["text_len"] = train_data["text"].apply(lambda s: len(s))
train_data["hashtags_count"] = train_data["hashtags"].apply(lambda s: s[1:-1].count("\'")/2)
train_data["day"] = train_data["timestamp"].apply(lambda t: datetime.utcfromtimestamp(t/1000).day)
train_data["hour"] = train_data["timestamp"].apply(lambda t: datetime.utcfromtimestamp(t/1000).hour)

# indicators of keywords
train_data["Macron"] =  train_data["text"].apply(lambda s: ("macron" in s.lower().split()))
train_data["Zemmour"] =  train_data["text"].apply(lambda s: ("zemmour" in s.lower().split()))
train_data["Melenchon"] =  train_data["text"].apply(lambda s: ("melenchon" in s.replace("é","e").lower().split()))
train_data["rt"] =  train_data["text"].apply(lambda s: ("rt" in s.lower().split()))

In [5]:
# separate target
target = train_data["retweets_count"]
train_data = train_data.drop(["retweets_count"], axis=1)

In [6]:
# select useful columns
train_data_filtered = train_data.drop(["text", "urls", "mentions", "hashtags", "timestamp", "TweetID"], axis=1)

# Standardize the data
normal_columns = train_data_filtered.drop(["hour", "verified", "Macron", "Zemmour", "Melenchon", "url_count", "rt"], axis=1).columns
mu, sigma = train_data_filtered[normal_columns].mean(axis=0), train_data_filtered[normal_columns].std(axis=0)
train_data_filtered.loc[:, normal_columns] = (train_data_filtered[normal_columns] - mu) / sigma

In [7]:
display(train_data_filtered)

,favorites_count,followers_count,statuses_count,friends_count,verified,url_count,text_len,hashtags_count,day,hour,Macron,Zemmour,Melenchon,rt
0,-0.054757,-0.063622,3.575856,0.866468,0,0.0,-0.865385,-0.384248,-1.030050,5,True,False,False,True
1,-0.054757,-0.077459,-0.415125,-0.469565,0,0.0,-1.466000,-0.384248,1.385496,12,False,False,False,False
2,-0.053583,-0.070310,-0.175076,0.214033,0,0.0,-1.225754,-0.384248,0.177723,18,False,False,False,False
3,-0.054757,-0.077786,-0.414631,-0.583032,0,1.0,1.272804,-0.384248,-0.124220,11,False,False,False,False
4,-0.054757,-0.024083,-0.200855,3.748287,0,0.0,-1.057582,-0.384248,-0.124220,11,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
353964,-0.054757,-0.077659,-0.410777,-0.561057,0,1.0,-0.961483,-0.384248,0.479666,13,False,False,False,False
353965,-0.054757,-0.077448,-0.325607,-0.532291,0,0.0,-0.024524,-0.384248,-0.728107,8,False,False,False,False
353966,-0.054757,-0.070525,-0.417806,0.649922,0,0.0,-0.985508,-0.384248,1.083553,12,False,True,False,False
353967,-0.054757,-0.077255,-0.419799,-0.455182,0,0.0,-1.105631,-0.384248,-1.030050,8,False,False,False,False


In [8]:
emb_matrix = np.load("data/train_emb_matrix.npy")
print(train_data_filtered.shape, emb_matrix.shape)

(353969, 14) (353969, 20, 8)


In [9]:
data_newdim = np.expand_dims(np.array(train_data_filtered), axis=1)
data_reshaped = np.broadcast_to(data_newdim, (emb_matrix.shape[0], emb_matrix.shape[1], train_data_filtered.shape[1]))
data_reshaped[0][:2]

array([[-0.05475705537294453, -0.06362175216471, 3.575856294985138,
        0.8664677648458644, 0, 0.0, -0.865384899170238,
        -0.3842477354707097, -1.0300503271633217, 5, True, False, False,
        True],
       [-0.05475705537294453, -0.06362175216471, 3.575856294985138,
        0.8664677648458644, 0, 0.0, -0.865384899170238,
        -0.3842477354707097, -1.0300503271633217, 5, True, False, False,
        True]], dtype=object)

In [10]:
# full_matrix = data_reshaped

In [11]:
full_matrix = np.concatenate((data_reshaped, emb_matrix), axis=2)

In [12]:
print(full_matrix[0][:2])

[[-0.05475705537294453 -0.06362175216471 3.575856294985138
  0.8664677648458644 0 0.0 -0.865384899170238 -0.3842477354707097
  -1.0300503271633217 5 True False False True 0.2411593794822693
  -1.0092555284500122 -1.6876295804977417 0.7724331617355347
  -0.658387303352356 0.3222971558570862 0.825392484664917
  0.5634337067604065]
 [-0.05475705537294453 -0.06362175216471 3.575856294985138
  0.8664677648458644 0 0.0 -0.865384899170238 -0.3842477354707097
  -1.0300503271633217 5 True False False True 0.4071543514728546
  1.6306689977645874 -0.12804804742336273 -0.6038780808448792
  -0.807412326335907 -0.21964296698570251 0.4271683692932129
  -0.06367146968841553]]


In [13]:
# full_matrix = np.expand_dims(full_matrix, axis=(2, 1))
full_matrix = full_matrix[..., np.newaxis]

In [14]:
print(full_matrix.shape)

(353969, 20, 22, 1)


In [26]:
print(full_matrix[0][:2])

[[[-0.05475705537294453]
  [-0.06362175216471]
  [3.575856294985138]
  [0.8664677648458644]
  [0]
  [0.0]
  [-0.865384899170238]
  [-0.3842477354707097]
  [-1.0300503271633217]
  [5]
  [True]
  [False]
  [False]
  [True]
  [0.2411593794822693]
  [-1.0092555284500122]
  [-1.6876295804977417]
  [0.7724331617355347]
  [-0.658387303352356]
  [0.3222971558570862]
  [0.825392484664917]
  [0.5634337067604065]]

 [[-0.05475705537294453]
  [-0.06362175216471]
  [3.575856294985138]
  [0.8664677648458644]
  [0]
  [0.0]
  [-0.865384899170238]
  [-0.3842477354707097]
  [-1.0300503271633217]
  [5]
  [True]
  [False]
  [False]
  [True]
  [0.4071543514728546]
  [1.6306689977645874]
  [-0.12804804742336273]
  [-0.6038780808448792]
  [-0.807412326335907]
  [-0.21964296698570251]
  [0.4271683692932129]
  [-0.06367146968841553]]]


In [27]:
X_train, X_eval, y_train, y_eval = train_test_split(full_matrix, target.values, random_state=42, test_size=0.1)

In [28]:
print(X_train.shape, y_train.shape, X_eval.shape, y_eval.shape)

(318572, 20, 22, 1) (318572,) (35397, 20, 22, 1) (35397,)


In [29]:
filter = 200000

X_train = X_train[filter:]
y_train = y_train[filter:]

In [30]:
print(X_train.shape, y_train.shape, X_eval.shape, y_eval.shape)

(118572, 20, 22, 1) (118572,) (35397, 20, 22, 1) (35397,)


In [31]:
import tensorflow as tf

In [32]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout

n_timesteps = X_train.shape[1]
input_features = X_train.shape[2]

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same',
                            input_shape=(X_train.shape[1], X_train.shape[2], 1)))
model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same',
                            input_shape=(X_train.shape[1], X_train.shape[2], 1)))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(32, activation='relu', kernel_initializer='he_uniform'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='linear'))

In [ ]:
# model = Sequential()
# model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(32, 32, 3)))
# model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
# model.add(MaxPooling2D((2, 2)))
# model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
# model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
# model.add(MaxPooling2D((2, 2)))
# model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
# model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
# model.add(MaxPooling2D((2, 2)))
# model.add(Flatten())
# model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
# model.add(Dense(10, activation='softmax'))

In [21]:
# model = tf.keras.models.Sequential([
#   tf.keras.layers.Dense(64, activation='relu'),
#   tf.keras.layers.Dense(1),
# ])

In [22]:
# import tensorflow_addons as tfa
# optimizer = tfa.optimizers.AdamW(learning_rate=1e-3, weight_decay=1e-4)
# model.compile(optimizer=optimizer, loss='mae')

In [33]:
model.compile(optimizer="adam", loss='mae')

In [34]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 20, 22, 32)        320       
                                                                 
 conv2d_3 (Conv2D)           (None, 20, 22, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 10, 11, 32)       0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 3520)              0         
                                                                 
 dense_2 (Dense)             (None, 128)               450688    
                                                                 
 dense_3 (Dense)             (None, 1)                 129       
                                                      

In [35]:
history = model.fit(X_train.astype(np.float32), y_train.astype(np.float32), epochs=50, batch_size=256,
         validation_data=(X_eval.astype(np.float32), y_eval.astype(np.float32)), shuffle=True)

Epoch 1/50
464/464 [==============================] - 16s 34ms/step - loss: 12.5405 - val_loss: 7.8798
Epoch 2/50
464/464 [==============================] - 15s 33ms/step - loss: 7.5518 - val_loss: 7.6771
Epoch 3/50
464/464 [==============================] - 15s 32ms/step - loss: 7.4582 - val_loss: 7.2882
Epoch 4/50
464/464 [==============================] - 15s 32ms/step - loss: 7.2287 - val_loss: 7.0845
Epoch 5/50
464/464 [==============================] - 15s 32ms/step - loss: 6.9303 - val_loss: 7.1850
Epoch 6/50
464/464 [==============================] - 15s 32ms/step - loss: 6.7489 - val_loss: 6.8193
Epoch 7/50
464/464 [==============================] - 15s 32ms/step - loss: 6.6031 - val_loss: 6.7493
Epoch 8/50
464/464 [==============================] - 15s 33ms/step - loss: 6.4187 - val_loss: 6.6272
Epoch 9/50
464/464 [==============================] - 16s 34ms/step - loss: 6.3418 - val_loss: 7.1275
Epoch 10/50
464/464 [==============================] - 17s 38ms/step - loss: 6.10

In [ ]:
plt.plot(history.epoch, history.history["loss"], 'b', label='Training loss')
plt.plot(history.epoch, history.history["val_loss"], 'g', label='Validation loss')
plt.title('Training loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
model.evaluate(X_eval.astype(np.float32),  y_eval.astype(np.float32), verbose=2)

In [ ]:
pred = model.predict(X_train.astype(np.float32))
print(pred)

In [ ]:
# plt.scatter(X_train["favorites_count"], model.predict(X_train))
# plt.scatter(X_train["favorites_count"], y_train.values.astype(np.float32))

In [ ]:
# evaluation

eval_data = pd.read_csv("data/evaluation.csv")
tweets = eval_data["TweetID"]

In [ ]:
eval_data["url_count"] = eval_data["urls"].apply(lambda s: s[1:-1].count("\'")/2)
eval_data["text_len"] = eval_data["text"].apply(lambda s: len(s))
eval_data["hashtags_count"] = eval_data["hashtags"].apply(lambda s: s[1:-1].count("\'")/2)
eval_data["day"] = eval_data["timestamp"].apply(lambda t: datetime.utcfromtimestamp(t/1000).day)
eval_data["hour"] = eval_data["timestamp"].apply(lambda t: datetime.utcfromtimestamp(t/1000).hour)
eval_data["Macron"] =  eval_data["text"].apply(lambda s: ("macron" in s.lower().split()))
eval_data["Zemmour"] =  eval_data["text"].apply(lambda s: ("zemmour" in s.lower().split()))
eval_data["Melenchon"] =  eval_data["text"].apply(lambda s: ("melenchon" in s.lower().split()))
eval_data["rt"] =  eval_data["text"].apply(lambda s: ("rt" in s.lower().split()))

# print("sentiment analysis...")
# eval_data["compound"] =  eval_data["text"].apply(lambda s: sia.polarity_scores(s)['compound'])

eval_data = eval_data.drop(["text", "urls", "mentions", "hashtags", "timestamp", "TweetID"], axis=1)

# normalize
eval_data.loc[:, normal_columns] = (eval_data.loc[:, normal_columns] - mu) / sigma

print(eval_data)

pred = model.predict(eval_data.values.astype(np.float32))

print(pred)

# output normalization
for i,p in enumerate(pred):
    if p<0: pred[i] = 0

In [ ]:
with open("data/predictions.csv", 'w') as f:
    writer = csv.writer(f)
    writer.writerow(["TweetID", "retweets_count"])
    for index, prediction in enumerate(pred):
        writer.writerow([str(tweets[index]) , str(int(prediction))])